In [5]:
import docling
from importlib.metadata import version
import warnings
import os


os.environ["DOCLING_ALLOW_EXTERNAL_PLUGINS"] = "true"
warnings.filterwarnings("ignore")

print(version('docling'))

2.69.0


In [ ]:
from pathlib import Path
from docling.document_converter import DocumentConverter

src_path = Path("SRCs")
pdfs = list(src_path.glob("*.pdf"))

if not pdfs:
    raise FileNotFoundError("No PDFs found in SRCs")

converter = DocumentConverter()
docs = converter.convert(pdfs[0])





[PictureItem(self_ref='#/pictures/0', parent=RefItem(cref='#/body'), children=[RefItem(cref='#/texts/30'), RefItem(cref='#/texts/31'), RefItem(cref='#/texts/32'), RefItem(cref='#/texts/33'), RefItem(cref='#/texts/34'), RefItem(cref='#/texts/35'), RefItem(cref='#/texts/36'), RefItem(cref='#/texts/37'), RefItem(cref='#/texts/38'), RefItem(cref='#/texts/39'), RefItem(cref='#/texts/40'), RefItem(cref='#/texts/41'), RefItem(cref='#/texts/42'), RefItem(cref='#/texts/43'), RefItem(cref='#/texts/44'), RefItem(cref='#/texts/45'), RefItem(cref='#/texts/46'), RefItem(cref='#/texts/47'), RefItem(cref='#/texts/48'), RefItem(cref='#/texts/49'), RefItem(cref='#/texts/50'), RefItem(cref='#/texts/51'), RefItem(cref='#/texts/52'), RefItem(cref='#/texts/53'), RefItem(cref='#/texts/54'), RefItem(cref='#/texts/55'), RefItem(cref='#/texts/56'), RefItem(cref='#/texts/57'), RefItem(cref='#/texts/58'), RefItem(cref='#/texts/59'), RefItem(cref='#/texts/60'), RefItem(cref='#/texts/61'), RefItem(cref='#/texts/62'

In [ ]:
result=docs.document

print(len(result.pictures))
print(len(result.tables))



6
4


Various file formats

In [ ]:
document_content=docs.document

json='parser/doc.json'
markdown='parser/doc.md'
html='parser/doc.html'
markdown_content=document_content.export_to_markdown()

if not json:
    document_content.save_as_json(json)

if not html:
    document_content.save_as_html(html)

if not markdown:
    with open(markdown,'w') as file:
        file.write(markdown_content)

Markdown

In [14]:
doc=docs.document
markdown_path='parser/doc.md'
result=doc.export_to_markdown()
with open(markdown_path,'w') as file:
    file.write(result)


print(docs.status)
print(docs.input.format)
print(docs.input.file)
print(docs.input.document_hash[:16])


ConversionStatus.SUCCESS
InputFormat.PDF
SRCs/rag.pdf
396a0fadeb4cd40f


PDF

In [4]:
from docling.datamodel.base_models import InputFormat
from docling.datamodel.pipeline_options import PdfPipelineOptions
from docling.document_converter import DocumentConverter,PdfFormatOption
from pathlib import Path

pdf_pipeline=PdfPipelineOptions(
     do_ocr=False,              # Disable OCR for native PDFs (faster)
    do_table_structure=True,   # Enable table structure extraction
    generate_page_images=True, # Generate page images for HTML export

)

converter=DocumentConverter(
    format_options={
        InputFormat.PDF: PdfFormatOption(pipeline_options=pdf_pipeline)
    }
)
doc=list(Path('SRCs').glob('*.pdf'))[0]

docs=converter.convert(doc)

print(len(docs.document.tables))




4


Word

In [21]:
from docling.datamodel.base_models import InputFormat
from docling.document_converter import DocumentConverter,WordFormatOption

convertor=DocumentConverter(
    allowed_formats=[InputFormat.XLSX,InputFormat.PPTX,InputFormat.DOCX]
)


pdf_path=list(Path('SRCs').glob('*.xlsx'))[0]

result=convertor.convert(pdf_path)

print(result.document.export_to_markdown())

| workflow_name   | definition_type   | task_name   | execution_type   | step_name   | runtime   | script               | depends   | input_parameters   | script_type   | execution_type   |
|-----------------|-------------------|-------------|------------------|-------------|-----------|----------------------|-----------|--------------------|---------------|------------------|
| workflow        | yaml              | task1       | local            | step1       | python    | print("Hello World") | None      | None               | inline        | local            |
| workflow        | yaml              | task1       | local            | step2       | shell     | echo "Done"          | step1     | None               | inline        | local            |
| workflow        | yaml              | task2       | local            | stepA       | python    | print("Task2")       | None      | None               | inline        | local            |


Image

In [9]:
from docling.document_converter import DocumentConverter,ImageFormatOption
from docling.datamodel.base_models import InputFormat
from docling.datamodel.pipeline_options import PdfPipelineOptions

pdf_pipeline=PdfPipelineOptions(
    do_ocr=True
)

convertor=DocumentConverter(
    format_options={
        InputFormat.IMAGE:ImageFormatOption(pipeline_options=pdf_pipeline)
    }
)
doc=list(Path('SRCs').glob('*.pdf'))[0]
result=convertor.convert(doc)

path='parser/pic.html'

result.document.save_as_html(
    path        
)




PPT

In [6]:
from docling.document_converter import DocumentConverter,WordFormatOption
from docling.datamodel.base_models import InputFormat
from pathlib import Path
# from docling.datamodel.pipeline_options import 

convertor=DocumentConverter(
    allowed_formats=[InputFormat.PPTX,InputFormat.XLSX,InputFormat.DOCX]
)
doc=list(Path('SRCs').glob('*.pptx'))[0]
result=convertor.convert(doc)
path='parser/doc.html'
result.document.save_as_html(path)






OCRs. (For handwritten Things)

In [ ]:
from docling.datamodel.pipeline_options import (
    EasyOcrOptions,
    TesseractOcrOptions,
    RapidOcrOptions,
    PdfPipelineOptions
)

rapidOcr_pipeline=PdfPipelineOptions(
    do_ocr=True,
    ocr_options=RapidOcrOptions()
)

easyOcr_pipeline=PdfPipelineOptions(
    do_ocr=True,
    ocr_options=EasyOcrOptions(
        lang=["fr", "de", "es", "en"],
        use_gpu=False
    )
)

tesseractOcr_pipeline=PdfPipelineOptions(
    do_ocr=True,
    ocr_options=TesseractOcrOptions(
        lang=["eng","fra"]
    )
)

convertor=DocumentConverter(
    format_options={
        InputFormat.PDF:PdfFormatOption(pipeline_options=easyOcr_pipeline)
    }
)


Table Structure

In [ ]:
from docling.document_converter import DocumentConverter,PdfFormatOption
from docling.datamodel.base_models import InputFormat
from docling.datamodel.pipeline_options import PdfPipelineOptions,TableFormerMode,TableStructureOptions

table_structure=TableStructureOptions(
    do_cell_matching=True,
    mode=TableFormerMode.ACCURATE
)
excel_pipeline=PdfPipelineOptions(
    do_ocr=True,
    table_structure_options=table_structure
)
convertor=DocumentConverter(
    format_options={
        InputFormat.PDF:PdfFormatOption(pipeline_options=excel_pipeline)}
)
doc=list(Path('SRCs').glob('*.pdf'))[0]
docs=convertor.convert(doc)

result=docs.document.save_as_html('parser/doc.html')




Langchain Docling

In [13]:
from langchain_docling import DoclingLoader
from langchain_docling.loader import ExportType
from pathlib import Path
folder_path='SRCs'
file_path=list(Path(folder_path).glob('*.pdf'))[0]


loader=DoclingLoader(
    file_path=file_path,
    export_type=ExportType.DOC_CHUNKS
)

docs=loader.load()
print(docs)

Token indices sequence length is longer than the specified maximum sequence length for this model (4904 > 512). Running this sequence through the model will result in indexing errors


[Document(metadata={'source': PosixPath('SRCs/rag.pdf'), 'dl_meta': {'schema_name': 'docling_core.transforms.chunker.DocMeta', 'version': '1.0.0', 'doc_items': [{'self_ref': '#/texts/3', 'parent': {'$ref': '#/body'}, 'children': [], 'content_layer': 'body', 'label': 'text', 'prov': [{'page_no': 1, 'bbox': {'l': 63.416, 't': 666.987, 'r': 548.584, 'b': 642.411, 'coord_origin': 'BOTTOMLEFT'}, 'charspan': [0, 147]}]}, {'self_ref': '#/texts/4', 'parent': {'$ref': '#/body'}, 'children': [], 'content_layer': 'body', 'label': 'text', 'prov': [{'page_no': 1, 'bbox': {'l': 116.313, 't': 630.324, 'r': 119.719, 'b': 623.739, 'coord_origin': 'BOTTOMLEFT'}, 'charspan': [0, 1]}]}, {'self_ref': '#/texts/5', 'parent': {'$ref': '#/body'}, 'children': [], 'content_layer': 'body', 'label': 'text', 'prov': [{'page_no': 1, 'bbox': {'l': 99.04, 't': 628.506, 'r': 512.963, 'b': 592.399, 'coord_origin': 'BOTTOMLEFT'}, 'charspan': [0, 223]}]}], 'headings': ['Retrieval-Augmented Generation for Large Language Mo

Langchain Docling Save to Markdown

In [14]:
from langchain_docling import DoclingLoader
from langchain_docling.loader import ExportType
folder_path='SRCs'
file_path=list(Path(folder_path).glob('*.pdf'))[0]

loader=DoclingLoader(
    file_path=file_path,
    export_type=ExportType.MARKDOWN
)

docs=loader.load()

print(docs)

[Document(metadata={'source': PosixPath('SRCs/test.pdf')}, page_content="## Welcome to Smallpdf\n\nReady to take document management to the next level?\n\n## Digital Documents-All In One Place\n\nWith the new Smallpdf experience, you can freely upload, organize, and share digital documents. When you enable the 'Storage' option, we'll also store all processed files here.\n\n## Enhance Documents in One Click\n\nWhen you right-click on a file, we'll present you with an array of options to convert, compress, or modify it.\n\n## Collaborate With Others\n\nForget mundane administrative tasks. With Smallpdf, you can request e-signatures, send large files, or even enable the Smallpdf G Suite App for your entire organization.\n\n## Access Files Anytime, Anywhere\n\nYou can access files stored on Smallpdf from your computer, phone, or tablet. We'll also sync files from the Smallpdf Mobile App to our online portal")]


Langchain Docling Custom Loader

In [ ]:
from langchain_docling import DoclingLoader
from docling.datamodel.base_models import InputFormat
from docling.datamodel.pipeline_options import PdfPipelineOptions
from docling.document_converter import PdfFormatOption,DocumentConverter

pdf_pipeline=PdfPipelineOptions(
    do_ocr=True,
    do_table_structure=True
)

converter=DocumentConverter(
    format_options={
        InputFormat.PDF:PdfFormatOption(pipeline_options=pdf_pipeline)
    }
)

loader=DoclingLoader(
    file_path=file_path,
    converter=converter,
    export_type=ExportType.DOC_CHUNKS
)

docs=loader.load()
print(docs)

